# RTS Data Analyst take-home assignment

🔍 Understand the Objective
You need to:
- Recommend volume per content (i.e., how much to produce) for each of 5 themes: info, sport, musique, societe, humour
- Help business understand the function each theme serves:
acquisition, retention, or loyalty
- Communicate this with clear insights and visuals

# 📌 1. Import Libraries

In [ ]:
import gc
gc.collect()

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy.stats import zscore


import warnings
warnings.filterwarnings('ignore')

# 📂 2. Load Raw Data

## 2.1. Mesures_contenu_volume_audio_à_commander.csv

In [ ]:
# Load the CSV file
path_volume = "../data/Mesures_contenu_volume_audio_à_commander.csv"
metrics_df = pd.read_csv(path_volume, sep=';', encoding='utf-8')

# Show first few rows of each for context
metrics_df

In [ ]:
metrics_df.info()

In [ ]:
metrics_df.describe()

In [ ]:
metrics_df.isnull().sum()

## 2.2. Correspondance_show_segment_tag.csv

In [ ]:
# Load the CSV file
path_tags = "../data/Correspondance_show_segment_tag.csv"
tags_df = pd.read_csv(path_tags, sep=';', encoding='utf-8')

# Show rows for context
tags_df

In [ ]:
tags_df.info()

In [ ]:
tags_df.describe()

In [ ]:
tags_df.isnull().sum()

# 🧼 3. Clean and Prepare Metrics

#### 3.1. Cleaning titles

In [ ]:
# remove trailing spaces from column names
metrics_df.columns = metrics_df.columns.str.rstrip()
# remove extra characters
metrics_df.columns = metrics_df.columns.str.strip().str.replace(r'[^\x00-\x7F]+', '', regex=True)

# Renaming 'Segment Length' to 'Episode Length (s)' as per data description
metrics_df = metrics_df.rename(columns={'Segment Length': 'Episode Length (s)'})

#### 3.2. Checking duplicated rows

In [ ]:
# Count duplicated rows (full row duplicates)
duplicate_rows = metrics_df.duplicated()
print(duplicate_rows.sum())

#### 3.3. Missing values

In [ ]:
# Identify rows
## impact on 'Show ID','Show','Publication Date','App/Site Name', 'Device Class'
missing_rows_1 = metrics_df[metrics_df['Show ID'].isnull()]
missing_rows_2 = metrics_df[metrics_df['Publication Date'].isnull()]
missing_rows_3 = metrics_df[metrics_df['App/Site Name'].isnull()]
missing_rows_4 = metrics_df[metrics_df['Device Class'].isnull()]

## Checking if the missing rows are the same
missing_rows_dfs = [missing_rows_1, missing_rows_2, missing_rows_3, missing_rows_4]
for i, missing_rows in enumerate(missing_rows_dfs):
    i +=1
    missing_count = len(missing_rows)
    total_count = len(metrics_df)
    missing_ratio = missing_count / total_count
    print(f"missing_rows_{i}: {missing_count}, Total rows: {total_count}, Missing ratio: {missing_ratio:.2%}")

In [ ]:
# To see if the exact same rows are missing *only* when all these columns are null
## we will check for the intersection of the null masks: "Are these rows null IN ALL specified columns simultaneously?"
all_specified_cols_null_mask = (
    metrics_df['Show ID'].isnull() &
    metrics_df['Publication Date'].isnull() &
    metrics_df['App/Site Name'].isnull() &
    metrics_df['Device Class'].isnull()
)
rows_where_all_specified_are_missing = metrics_df[all_specified_cols_null_mask]


## Then we compare this combined result to our individual missing_rows_X DataFrames
for i, missing_rows in enumerate(missing_rows_dfs):
    i += 1
    print(f"Is missing_rows_{i} identical to rows where ALL specified columns are missing?",
          missing_rows.equals(rows_where_all_specified_are_missing))

Regarding the file "Mesures_contenu_volume_audio_à_commander.csv", I have found that 0.04% (119 rows) of the data have missing values on the exact same rows. Moreover, after further investigation, it was also found that these rows containing missing values also have wrong formatting/value in their numerical inputs and wrong titles. Consequently, I will remove the rows containing missing values instead of placig a placeholder

In [ ]:
# drop the rows containing empty values in 'Show ID', 'Publication Date', 'App/Site Name', 'Device Class'
metrics_df = metrics_df.dropna(subset=['Show ID', 'Publication Date', 'App/Site Name', 'Device Class']).copy()
print(f"Remaining entries after drop: {metrics_df.shape[0]}")

#### 3.4. Data Consistency Checks

In [ ]:
metrics_df.columns

- "New Visit Rate %" column

In [ ]:
# Noticed that many values were formatted with a coma and a '%' sign

## Converting the column to string
metrics_df["New Visit Rate %"] = metrics_df["New Visit Rate %"].astype(str)

## Remove the '%' character and replace ',' with '.' for decimal conversion
metrics_df["New Visit Rate %"] = metrics_df["New Visit Rate %"] \
                                 .str.replace('%', '', regex=False) \
                                 .str.replace(',', '.', regex=False)

- "Publication Date" column

In [ ]:
# Date standardization based on "DD.MM.YYYY"
metrics_df['Publication Date'] = pd.to_datetime(metrics_df['Publication Date'], format='%d.%m.%Y', errors='coerce')
# metrics_df['Publication Date']

- Converting time strings to seconds

In [ ]:
# Function to convert hh:mm:ss to total seconds
def duration_to_seconds(duration_str):
    try:
        h, m, s = map(int, duration_str.split(':'))
        return h * 3600 + m * 60 + s
    except:
        return None  # Handles invalid formats

# Apply conversion to 'Avg Play Duration'
metrics_df['Avg Play Duration (s)'] = metrics_df['Avg Play Duration'].apply(duration_to_seconds)
# metrics_df['Avg Play Duration (s)']

# Apply conversion to 'Total Play Duration'
metrics_df['Total Play Duration (s)'] = metrics_df['Total Play Duration'].apply(duration_to_seconds)
# metrics_df['Total Play Duration (s)']

- Converting numerical values

In [ ]:
# Numeric columns check
numeric_columns = ['Episode Length (s)', 'Media Views', 'Visitors', 'New Visit Rate %', 
                   'Entries', 'Exits', 'Returning Visits', 'Bounces',
                   'Avg Play Duration (s)', 'Total Play Duration (s)']
# Ensure columns are converted to float explicitly
metrics_df[numeric_columns] = metrics_df[numeric_columns].apply(lambda col: pd.to_numeric(col, errors='coerce')).astype(float)

# Check for numeric conversion issues
print("Numeric conversion check:")
print(metrics_df[numeric_columns].isnull().sum())

- Converting categorical values

In [ ]:
# Convert selected columns to categorical type
categorical_columns = ['Segment ID', 'Segment', 'Show ID', 'Show', 'App/Site Name', 'Device Class']
metrics_df[categorical_columns] = metrics_df[categorical_columns].astype('category')

In [ ]:
metrics_df

In [ ]:
# Dropping the column "Avg Play Duration" & "Total Play Duration" 
# as we have their values in seconds in "Avg Play Duration (s)" & "Total Play Duration (s)"
metrics_df.drop(columns=['Avg Play Duration', 'Total Play Duration'], inplace=True)

In [ ]:
metrics_df.info()

In [ ]:
# Count duplicated rows (full row duplicates)
duplicate_rows = metrics_df.duplicated()
print(duplicate_rows.sum())

#### 3.5 Checking outliers

In [ ]:
# checking the statistics of the numerical values
metrics_df[numeric_columns].describe().T

In [ ]:
# checking the distribution
metrics_df[numeric_columns].plot(kind='box', subplots=True, layout=(3,4), figsize=(15,10), sharex=False)
plt.tight_layout()
plt.show()

In [ ]:
# statistical check: Z-Score Method

## Z-scores for numeric columns
z_scores = metrics_df[numeric_columns].apply(zscore)
outlier_flags = (z_scores.abs() > 3)  # mark outliers (threshold=3 std deviations)

## Check which columns have many outliers
outlier_counts = outlier_flags.sum()
print(outlier_counts)


In [ ]:
# Cap at 99th Percentile
## to reduce outlier impact
for col in numeric_columns:
    cap = metrics_df[col].quantile(0.99)
    metrics_df[col] = metrics_df[col].clip(upper=cap)

# 🧼 4. Clean and Prepare Tags

#### 4.1 Cleaning titles

In [ ]:
# Clean column names (e.g., remove invisible characters)
tags_df.columns = tags_df.columns.str.strip().str.replace(r'[^\x00-\x7F]+', '', regex=True)

#### 4.2. Cleaning 'Assigned Tags'

In [ ]:
# Replace '-' with None
tags_df['Assigned Tags'] = tags_df['Assigned Tags'].replace('-', None)

# Drop rows where 'Assigned Tags' is None or effectively empty after stripping whitespace
tags_df = tags_df[tags_df['Assigned Tags'].notna() & (tags_df['Assigned Tags'].astype(str).str.strip() != '')]

In [ ]:
# Define the five exact valid tags
valid_tags = {
    'media_radio:societe',
    'media_radio:humour',
    'media_radio:info',
    'media_radio:musique',
    'media_radio:sport'
}

In [ ]:
# Function to check if each valid tag is present in the full string
def match_valid_tags_in_string(tag_string, valid_tags):
    tag_string = str(tag_string).lower()
    return [tag for tag in valid_tags if tag in tag_string]

In [ ]:
# Apply the matching function
tags_df['cleaned_themes'] = tags_df['Assigned Tags'].apply(lambda x: match_valid_tags_in_string(x, valid_tags))
tags_df['Primary Theme'] = tags_df['cleaned_themes'].apply(lambda tags: tags[0] if tags else None)

We noticed that there are 10 shows that contain multi themes:

    3ème mi-temps
    Dis, pourquoi?
    Émission spéciale
    Footaises
    La Matinale
    Le 12h30
    Le grand soir
    Les beaux parleurs
    Sport-Première
    The Jam

In this particular case, we are sticking to the assumption of primary_theme = first theme.
Otherwise we could explode to multi-theme rows, but shows could be repeated among categories.

#### 4.3 Dropping null values in 'Primary Theme'

In [ ]:
# Drop all the empty 'Primary Theme' as we could not retrieve the needed ones
tags_df = tags_df.dropna(subset=['Primary Theme'])

#### 4.4 Dropping unnecessary columns

In [ ]:
# Drop column 'Assigned Tags' and 'cleaned themes' as it was to check the extractions
tags_df = tags_df.drop(["Assigned Tags", "cleaned_themes"], axis=1)

#### 4.5. Checking duplicated rows

In [ ]:
# Remove duplicated rows and keeping the first time the tag was assigned
tags_df = tags_df.drop_duplicates(subset=['Segment ID', 'Show', 'Show ID'], keep='first')

In [ ]:
tags_df

In [ ]:
# tags_df.to_csv("tags.csv", encoding='utf-8-sig')

# 🔗 5. Merge Datasets

#### 5.1 Merge datasets

In [ ]:
# Merge datasets on "Segment ID"
# many-to-one method as we have multiple Segment ID due to "App/Site Name" & "Device Class"
merged_df = pd.merge(metrics_df, tags_df[['Segment ID', 'Primary Theme']],
                     on='Segment ID', how='left')

# Check merge results
merged_df

#### 5.2. Checking duplicates after merging

In [ ]:
# Count duplicated rows (full row duplicates)
merged_df.sort_values('Segment ID')
print(merged_df.duplicated(keep=False).sum())

# merged_df.to_csv("dups.csv", encoding='utf-8-sig')

#### 5.3. Filtering valid tags

In [ ]:
# Dropping the rows that do not have the 5 tags that we needed to have
## valid tags from before. Applying this method if we were to add more valid_tags.
df = merged_df[merged_df['Primary Theme'].isin(valid_tags)]

# Remove 'media_radio:' prefix from 'Primary Theme'
df['Primary Theme'] = df['Primary Theme'].str.replace('media_radio:', '', regex=False)

## changing the values to categories for consistency
df['Primary Theme'] = df['Primary Theme'].astype('category')

## display data
df

#### 5.4. Define the KPI within the dataset

For each "Segment ID", we’ll calculate:

| KPI                           | Formula                          |
| ----------------------------- | -------------------------------- |
| 🔁 Engagement per Visitor     | `Total Play Duration / Visitors` |
| 🧲 Acquisition Rate           | `Entries / Visitors`             |
| 📌 Retention Rate             | `Returning Visits / Visitors`    |


In [ ]:
df["Engagement per Visitor (s)"] = df["Total Play Duration (s)"] / df["Visitors"].replace(0, pd.NA)
df["Acquisition Rate %"] = df["Entries"] / df["Visitors"].replace(0, pd.NA) * 100
df["Retention Rate %"] = df["Returning Visits"] / df["Visitors"].replace(0, pd.NA) * 100

#### 5.5. Exporting final dataset

In [ ]:
# Generating a file to keep for further investigation
df.to_csv("../data/rts_data_metrics_tags.csv", encoding='utf-8-sig', index=False)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df

# 📊 6. Compute KPIs per Theme

#### 6.1. Define KPIs

In [ ]:
# Convert numeric columns to ensure they aggregate properly
kpi_columns = [
    'Media Views', 'Visitors', 'New Visit Rate %', 'Entries', 'Exits',
    'Returning Visits', 'Bounces', 'Avg Play Duration (s)', 'Total Play Duration (s)'
]
df[kpi_columns] = df[kpi_columns].apply(pd.to_numeric, errors='coerce')

For each "Primary Theme", we’ll calculate:

| KPI                           | Formula                          |
| ----------------------------- | -------------------------------- |
| 🎧 Total Media Views          | `sum(Media Views)`               |
| 👥 Total Visitors             | `sum(Visitors)`                  |
| 📈 Average New Visit Rate (%) | `mean(New Visit Rate %)`         |
| 🚪 Total Entries              | `sum(Entries)`                   |
| 🚶‍♂️ Total Exits             | `sum(Exits)`                     |
| 🔁 Total Returning Visits     | `sum(Returning Visits)`          |
| ⛔️ Total Bounces              | `sum(Bounces)`                   |
| ⏱️ Average Play Duration (s)  | `mean(Avg Play Duration (s))`    |
| ⏱️ Total Play Duration (s)    | `sum(Total Play Duration (s))`   |
| 🔁 Engagement per Visitor     | `Total Play Duration / Visitors` |
| 🧲 Acquisition Rate           | `Entries / Visitors`             |
| 📌 Retention Rate             | `Returning Visits / Visitors`    |


In [ ]:
# Group by Primary Theme and compute aggregations
theme_kpis = df.groupby('Primary Theme').agg({
    'Media Views': 'sum',
    'Visitors': 'sum',
    'New Visit Rate %': 'mean',
    'Entries': 'sum',
    'Exits': 'sum',
    'Returning Visits': 'sum',
    'Bounces': 'sum',
    'Avg Play Duration (s)': 'mean',
    'Total Play Duration (s)': 'sum'
}).reset_index()

#### 6.2. Define KPI calculations

- Engagement per Visitor = Total Play Duration / Visitors
- Acquisition Rate = Entries / Visitors
- Retention Rate = Returning Visits / Visitors

In [ ]:
# Add calculated KPIs
theme_kpis['Engagement per Visitor'] = theme_kpis['Total Play Duration (s)'] / theme_kpis['Visitors'].replace(0, pd.NA)
theme_kpis['Acquisition Rate'] = theme_kpis['Entries'] / theme_kpis['Visitors'].replace(0, pd.NA)
theme_kpis['Retention Rate'] = theme_kpis['Returning Visits'] / theme_kpis['Visitors'].replace(0, pd.NA)

#### 6.3. Normalizing the KPIs

In [ ]:
# Min-max normalization (optional)
for col in ['Engagement per Visitor', 'Acquisition Rate', 'Retention Rate']:
    theme_kpis[f'norm_{col}'] = (theme_kpis[col] - theme_kpis[col].min()) / (theme_kpis[col].max() - theme_kpis[col].min())

#### 6.4. Recommendation system

In [ ]:
thresholds = {
    'high_engagement': theme_kpis['Engagement per Visitor'].quantile(0.75),
    'low_engagement': theme_kpis['Engagement per Visitor'].quantile(0.25),
    'high_retention': theme_kpis['Retention Rate'].quantile(0.75),
    'high_acquisition': theme_kpis['Acquisition Rate'].quantile(0.75),
}

In [ ]:
def recommend(row):
    # High retention + high engagement
    if (row['Engagement per Visitor'] >= thresholds['high_engagement'] and
        row['Retention Rate'] >= thresholds['high_retention']):
        return "Increase production"
    # High acquisition, moderate engagement
    elif (row['Acquisition Rate'] >= thresholds['high_acquisition'] and
          thresholds['low_engagement'] < row['Engagement per Visitor'] < thresholds['high_engagement']):
        return "Maintain"
    # Low engagement
    elif row['Engagement per Visitor'] <= thresholds['low_engagement']:
        return "Review and optimize"
    # All other cases
    else:
        return "Cross-promote"


#### 6.5. Export Segment-Level KPIs

In [ ]:
theme_kpis['Recommendation'] = theme_kpis.apply(recommend, axis=1)

In [ ]:
theme_kpis

In [ ]:
# Generating a file to keep for further investigation
theme_kpis.to_csv("../data/rts_theme_kpis.csv", encoding='utf-8-sig', index=False)

# 📊 7. Compute KPIs per Segment ID

In [ ]:
# Needed this extra step due to computing capabilities
import gc
gc.collect()

import pandas as pd


import warnings
warnings.filterwarnings('ignore')

# Generating a file to keep for further investigation
df = pd.read_csv("../data/rts_data_metrics_tags.csv", encoding='utf-8-sig')

#### 7.1. Group by 'Segment ID'

In [ ]:
segment_kpis = df.groupby(['Segment ID', 'Primary Theme']).agg({
    'Media Views': 'sum',
    'Visitors': 'sum',
    'New Visit Rate %': 'mean',
    'Entries': 'sum',
    'Exits': 'sum',
    'Returning Visits': 'sum',
    'Bounces': 'sum',
    'Avg Play Duration (s)': 'mean',
    'Total Play Duration (s)': 'sum'
}).reset_index()


#### 7.2. Add Calculated KPIs

In [ ]:
segment_kpis["Engagement per Visitor"] = segment_kpis["Total Play Duration (s)"] / segment_kpis["Visitors"].replace(0, pd.NA)
segment_kpis["Acquisition Rate"] = segment_kpis["Entries"] / segment_kpis["Visitors"].replace(0, pd.NA)
segment_kpis["Retention Rate"] = segment_kpis["Returning Visits"] / segment_kpis["Visitors"].replace(0, pd.NA)


#### 7.3. Normalize for Segment-Level Thresholds

In [ ]:
for col in ['Engagement per Visitor', 'Acquisition Rate', 'Retention Rate']:
    segment_kpis[f'norm_{col}'] = (segment_kpis[col] - segment_kpis[col].min()) / (segment_kpis[col].max() - segment_kpis[col].min())

#### 7.4. Recommendation

In [ ]:
thresholds_segment = {
    'high_engagement': segment_kpis['Engagement per Visitor'].quantile(0.75),
    'low_engagement': segment_kpis['Engagement per Visitor'].quantile(0.25),
    'high_retention': segment_kpis['Retention Rate'].quantile(0.75),
    'high_acquisition': segment_kpis['Acquisition Rate'].quantile(0.75),
}

In [ ]:
def recommend_segment(row):
    if (row['Engagement per Visitor'] >= thresholds_segment['high_engagement'] and
        row['Retention Rate'] >= thresholds_segment['high_retention']):
        return "Increase production"
    elif (row['Acquisition Rate'] >= thresholds_segment['high_acquisition'] and
          thresholds_segment['low_engagement'] < row['Engagement per Visitor'] < thresholds_segment['high_engagement']):
        return "Maintain"
    elif row['Engagement per Visitor'] <= thresholds_segment['low_engagement']:
        return "Review and optimize"
    else:
        return "Cross-promote"

segment_kpis['Recommendation'] = segment_kpis.apply(recommend_segment, axis=1)


#### 7.5. Export Segment-Level KPIs

In [ ]:
segment_kpis.to_csv("../data/rts_segment_kpis.csv", encoding='utf-8-sig', index=False)